# Preparing data for uploading to database
### I use pandas for easy manipulation

In [74]:
import pandas as pd
import numpy as np
import datetime


# Load csv into Dataframe
filename = 'epg.csv'
df = pd.read_csv(filename)
df.head(1)

,channel_id,channel_name,channel_country,start_date,start_time,duration_in_seconds,program_id,program_original_title,program_local_title,program_year
0,384303590,HD1 (France),FRA,20180101,00:00,84000,461044,À communiquer,À communiquer,NaN


In [75]:
channel_ids = df['channel_id'].unique()
# [384303590,  69036190,  69036687]
channel_names = df['channel_name'].unique()
# ['HD1 (France)', 'La Sexta', 'RTE 1']
channel_countries = df['channel_country'].unique() 
# ['FRA', 'ESP', 'IRL']
program_ids = df['program_id'].unique().size
# 1879
program_original_title = df['program_original_title'].unique().size
# 358
program_local_title = df['program_local_title'].unique().size
# 358

# What is the shape of the df
print("The data is {} cols by {} rows.".format(df.shape[1], df.shape[0]))
for column in df.columns:
    print(column, df[column].count())

print('-'*31)  
print('I can tell there are cells in "program_original_title" and "program_year" that are empty or NaN')

print('The quantity of unique values in each col are the following:')
for column in df.columns:
    print(column, df[column].unique().size)

The data is 10 cols by 4897 rows.
('channel_id', 4897)
('channel_name', 4897)
('channel_country', 4897)
('start_date', 4897)
('start_time', 4897)
('duration_in_seconds', 4897)
('program_id', 4897)
('program_original_title', 4860)
('program_local_title', 4897)
('program_year', 2464)
-------------------------------
I can tell there are cells in "program_original_title" and "program_year" that are empty or NaN
The quantity of unique values in each col are the following:
('channel_id', 3)
('channel_name', 3)
('channel_country', 3)
('start_date', 59)
('start_time', 266)
('duration_in_seconds', 98)
('program_id', 1879)
('program_original_title', 358)
('program_local_title', 372)
('program_year', 44)


### Fill null titles and years with n/a. 
```python
df = df.fillna('n/a)
```

In [76]:
df = df.fillna('n/a')
df.dtypes

channel_id                 int64
channel_name              object
channel_country           object
start_date                 int64
start_time                object
duration_in_seconds        int64
program_id                 int64
program_original_title    object
program_local_title       object
program_year              object
dtype: object

In [77]:
# Change DataFrame dtypes to match Database models
def changeDtypesForDb(df):
    df['program_year'] = pd.to_numeric(df['program_year'], errors='coerce')
    df['start_date'] = pd.to_datetime(df['start_date'], format='%Y%m%d', errors='ignore')
    df['start_time'] = df['start_time'].map(lambda x: datetime.datetime.strptime(x, '%H:%M').time())
    df['duration_in_seconds'] = pd.to_timedelta(df['duration_in_seconds'], unit='seconds')
    return df

In [78]:
# Add the df Showtimes to DB
df = changeDtypesForDb(df)
df.dtypes

channel_id                          int64
channel_name                       object
channel_country                    object
start_date                 datetime64[ns]
start_time                         object
duration_in_seconds       timedelta64[ns]
program_id                          int64
program_original_title             object
program_local_title                object
program_year                      float64
dtype: object

In [79]:
from tv import models
for object in models.Program.objects.all():
    object.delete()
for object in models.Channel.objects.all():
    object.delete()
for object in models.Showtime.objects.all():
    object.delete()

In [80]:
from django.db import IntegrityError
from tv import models


# Create Program with channel obj and row data
def createProgram(row, channel):
    try:
        program = models.Program(uid=row.program_id, 
                                 original_title=row.program_original_title,
                                 local_title=row.program_local_title, 
                                 year=row.program_year,
                                 channel=channel)
        program.save()
        print("Unique Program {} with id {} added to DB".format(program.local_title, program.uid))
        return program
    except IntegrityError:
        print("Program with unique id {} already exists".format(program.uid))
        return
    

# Create Showtime with program obj and row data
def createShowtime(row, program):
    try:
        showtime = models.Showtime(start_date=row.start_date, start_time=row.start_time,
                                           duration=row.duration_in_seconds, program=program)
        showtime.save()
        print('Showtime: {} on {} was added to DB.'.format(showtime.start_date,program.local_title))
        return showtime
    except IntegrityError:
        print("Showtime already exists in DB".format(showtime.pk))
        return
    
              
# Create Showtimes from DataFrame           
def addShowtimeFromDataFrame(df):
    for index, row in df.iterrows():
        channel, create = models.Channel.objects.get_or_create(uid=row.channel_id, 
                                                               name=row.channel_name,
                                                               country_code=row.channel_country)
        if not create:
            program = createProgram(row, channel)
            showtime = createShowtime(row, program)
        else:
            channel.name = row.channel_name
            channel.country_code = row.channel_country
            channel.save()
            program = createProgram(row, channel)
            showtime = createShowtime(row, program)
    print("Finished adding programs from {}".format(filename))
    return showtime
        


# Add to DB
```python
addShowtimeFromDataFrame(df)
```

In [81]:
addShowtimeFromDataFrame(df)
df.isnull()

Unique Program À communiquer with id 461044 added to DB
Showtime: 2018-01-01 00:00:00 on À communiquer was added to DB.
Unique Program Será anunciado with id 1887442 added to DB
Showtime already exists in DB
Unique Program RTE 1 Programming with id 3452333967 added to DB
Showtime already exists in DB
Unique Program L.A. Confidential with id 446381 added to DB
Showtime: 2018-01-01 00:00:00 on L.A. Confidential was added to DB.
Unique Program No controles with id 18063328 added to DB
Showtime: 2018-01-01 00:00:00 on No controles was added to DB.
Unique Program Canal Ruleta with id 5072179538 added to DB
Showtime: 2018-01-01 00:00:00 on Canal Ruleta was added to DB.
Unique Program Minutos musicales with id 11742116 added to DB
Showtime: 2018-01-01 00:00:00 on Minutos musicales was added to DB.
Unique Program Papillon with id 1064550 added to DB
Showtime already exists in DB
Program with unique id 11742116 already exists
Showtime already exists in DB
Unique Program Bestial with id 19219963

Unique Program Las primeras 48 horas with id 4804635922 added to DB
Showtime already exists in DB
Unique Program Au nom de la vérité with id 5601168087 added to DB
Showtime: 2018-01-02 00:00:00 on Au nom de la vérité was added to DB.
Unique Program Al Rojo Vivo with id 5668214430 added to DB
Showtime: 2018-01-02 00:00:00 on Al Rojo Vivo was added to DB.
Unique Program Au nom de la vérité with id 5542565692 added to DB
Showtime: 2018-01-02 00:00:00 on Au nom de la vérité was added to DB.
Unique Program Au nom de la vérité with id 5569923357 added to DB
Showtime: 2018-01-02 00:00:00 on Au nom de la vérité was added to DB.
Unique Program Au nom de la vérité with id 1156185999 added to DB
Showtime: 2018-01-02 00:00:00 on Au nom de la vérité was added to DB.
Unique Program Shortland Street with id 5612079579 added to DB
Showtime: 2018-01-02 00:00:00 on Shortland Street was added to DB.
Unique Program Au nom de la vérité with id 5547041841 added to DB
Showtime: 2018-01-02 00:00:00 on Au nom 

Program with unique id 17875416 already exists
Showtime already exists in DB
Program with unique id 12766064 already exists
Showtime already exists in DB
Program with unique id 12039424 already exists
Showtime already exists in DB
Program with unique id 11989490 already exists
Showtime already exists in DB
Program with unique id 17875409 already exists
Showtime already exists in DB
Unique Program Reeling in the Years with id 12068169 added to DB
Showtime: 2018-01-03 00:00:00 on Reeling in the Years was added to DB.
Unique Program Grey's Anatomy with id 12887841 added to DB
Showtime: 2018-01-03 00:00:00 on Grey's Anatomy was added to DB.
Program with unique id 319923221 already exists
Showtime already exists in DB
Unique Program Nationwide with id 5612079557 added to DB
Showtime already exists in DB
Unique Program Neven's Food Trails: The Basque Country with id 4089228914 added to DB
Showtime: 2018-01-03 00:00:00 on Neven's Food Trails: The Basque Country was added to DB.
Program with u

Showtime: 2018-01-04 00:00:00 on EastEnders was added to DB.
Program with unique id 12044548 already exists
Showtime already exists in DB
Program with unique id 5599159988 already exists
Showtime already exists in DB
Unique Program Fair City with id 5612079573 added to DB
Showtime: 2018-01-04 00:00:00 on Fair City was added to DB.
Unique Program V pour Vendetta with id 2039561 added to DB
Showtime already exists in DB
Program with unique id 11922093 already exists
Showtime already exists in DB
Unique Program Ear to the Ground with id 5612079537 added to DB
Showtime: 2018-01-04 00:00:00 on Ear to the Ground was added to DB.
Unique Program El intermedio Christmas Edition with id 5668214454 added to DB
Showtime already exists in DB
Program with unique id 12147320 already exists
Showtime already exists in DB
Unique Program Dos madres perfectas with id 666254531 added to DB
Showtime: 2018-01-04 00:00:00 on Dos madres perfectas was added to DB.
Unique Program Daddy's Home with id 863726785 a

Showtime: 2018-01-05 00:00:00 on Scannal was added to DB.
Program with unique id 12044548 already exists
Showtime already exists in DB
Program with unique id 5599159988 already exists
Showtime already exists in DB
Unique Program Pat Shortt's Music From D'Telly with id 1982269411 added to DB
Showtime: 2018-01-05 00:00:00 on Pat Shortt's Music From D'Telly was added to DB.
Unique Program Nos chers voisins with id 4442942482 added to DB
Showtime already exists in DB
Program with unique id 11922093 already exists
Showtime already exists in DB
Unique Program Francis Brennan's Grand Vietnamese Tour with id 5033379015 added to DB
Showtime: 2018-01-05 00:00:00 on Francis Brennan's Grand Vietnamese Tour was added to DB.
Unique Program #laSexta Columna with id 22365312 added to DB
Showtime already exists in DB
Program with unique id 12147320 already exists
Showtime already exists in DB
Unique Program Equipo de investigación with id 21714748 added to DB
Showtime: 2018-01-05 00:00:00 on Equipo de 

...

Program with unique id 4098771666 already exists
Showtime already exists in DB
Program with unique id 4120301248 already exists
Showtime already exists in DB
Program with unique id 218925 already exists
Showtime already exists in DB
Program with unique id 21714748 already exists
Showtime already exists in DB
Program with unique id 4098775249 already exists
Showtime already exists in DB
Program with unique id 4528725581 already exists
Showtime already exists in DB
Program with unique id 4098771664 already exists
Showtime already exists in DB
Unique Program Mass with id 5837038271 added to DB
Showtime: 2018-02-25 00:00:00 on Mass was added to DB.
Program with unique id 4120301230 already exists
Showtime already exists in DB
Unique Program Petits secrets entre voisins with id 4142698526 added to DB
Showtime: 2018-02-25 00:00:00 on Petits secrets entre voisins was added to DB.
Program with unique id 11914776 already exists
Showtime already exists in DB
Program with unique id 4142698524 alr

Unique Program Más vale tarde with id 5835149689 added to DB
Showtime: 2018-02-26 00:00:00 on Más vale tarde was added to DB.
Program with unique id 22271793 already exists
Showtime already exists in DB
Program with unique id 19733321 already exists
Showtime already exists in DB
Program with unique id 12766064 already exists
Showtime already exists in DB
Program with unique id 22301334 already exists
Showtime already exists in DB
Program with unique id 12039424 already exists
Showtime already exists in DB
Program with unique id 11989490 already exists
Showtime already exists in DB
Unique Program Grey's Anatomy with id 22301333 added to DB
Showtime: 2018-02-26 00:00:00 on Grey's Anatomy was added to DB.
Unique Program Nationwide with id 5837038273 added to DB
Showtime: 2018-02-26 00:00:00 on Nationwide was added to DB.
Program with unique id 319923221 already exists
Showtime already exists in DB
Unique Program Follow Donal...to Europe with id 3522350440 added to DB
Showtime: 2018-02-26 

Program with unique id 12039424 already exists
Showtime already exists in DB
Program with unique id 11989490 already exists
Showtime already exists in DB
Unique Program Grey's Anatomy with id 22335737 added to DB
Showtime: 2018-02-27 00:00:00 on Grey's Anatomy was added to DB.
Unique Program eco.eye with id 5837038253 added to DB
Showtime: 2018-02-27 00:00:00 on eco.eye was added to DB.
Program with unique id 319923221 already exists
Showtime already exists in DB
Unique Program EastEnders with id 5837433998 added to DB
Showtime: 2018-02-27 00:00:00 on EastEnders was added to DB.
Program with unique id 12044548 already exists
Showtime already exists in DB
Unique Program Guépardes with id 5823541818 added to DB
Showtime: 2018-02-27 00:00:00 on Guépardes was added to DB.
Program with unique id 5599159988 already exists
Showtime already exists in DB
Unique Program Fair City with id 5837038257 added to DB
Showtime: 2018-02-27 00:00:00 on Fair City was added to DB.
Unique Program Sissi impér

Unique Program Marseille with id 5827250903 added to DB
Showtime: 2018-02-28 00:00:00 on Marseille was added to DB.
Unique Program Operation Transformation with id 5837038307 added to DB
Showtime: 2018-02-28 00:00:00 on Operation Transformation was added to DB.
Program with unique id 12579138 already exists
Showtime already exists in DB
Unique Program The Classroom Divide with id 5259959914 added to DB
Showtime: 2018-02-28 00:00:00 on The Classroom Divide was added to DB.
Program with unique id 13121929 already exists
Showtime already exists in DB
Program with unique id 643777741 already exists
Showtime already exists in DB
Program with unique id 13071731 already exists
Showtime already exists in DB
Program with unique id 13071731 already exists
Showtime already exists in DB
Finished adding programs from epg.csv


,channel_id,channel_name,channel_country,start_date,start_time,duration_in_seconds,program_id,program_original_title,program_local_title,program_year
0,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,True
6,False,False,False,False,False,False,False,False,False,True
7,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,True
9,False,False,False,False,False,False,False,False,False,True


In order to make this data accessible to the rest of the development teams we need you to
develop an HTTP API consisting of one endpoint that accepts as input:

- start date, end date
- country (optional)
- title (optional)

and returns as output a list of the TV programs with their broadcasting data (channel and
time details) aired within the specified time interval and country, and matching the specified
title.

The list must be in JSON format. For example:

[
{
"program_id": "12044548",
"program_original_title": "Petits plats en équilibre","program_year": null,
"start_time": "20180101 - 13:00",
"end_time": "20180101 - 13:05",
...
},
{
"channel_id": "11939294",
"channel_name": "RTÉ News: One O'Clock",
"program_year": 2001,
"start_time": "20180101 - 13:00",
"end_time": "20180101 - 13:50",
},
...
]

In [ ]:
import urlparse

tv_api_endpoint = 'http://localhost:8080/tv/api/v1/'

params = {
start_date:'YYYY-MM-DD'
end_date='YYYY-MM-DD'
country='Spaìn'
title="Frank's back"}
query_string = '?start_date=YYYY-MM-DD&end_date=YYYY-MM-DD&title=SOME TITLE&country=COUNTRY'

url http://localhost:8080/api/tv/programs?start_date=YYYY-MM-DD&end_date=YYYY-MM-DD&title=SOME

add boolean check to put date parameters in correct order


In [ ]:
params = urllib.unquote(url)

In [ ]:
params.decode('utf-8')

help(urlparse)